### Convert COCO to YOLO for object detection - Not working

In [1]:
from ultralytics.data.converter import convert_coco

In [2]:
segments_base_folder = '/home/etaylor/code_projects/thesis/segments'
segments_folder_name = ""

annotations_dir = '/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_week9_15_06_2023_3x_regular_IMG_2157/annotations'
output_dir = "/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_week9_15_06_2023_3x_regular_IMG_2157/annotations/yolo"

In [3]:
convert_coco(
    labels_dir=annotations_dir,
    save_dir=output_dir, 
    use_segments=False,
)

Annotations /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_week9_15_06_2023_3x_regular_IMG_2157/annotations/export_coco-instance_etaylor_cannabis_patches_week9_15_06_2023_3x_regular_IMG_2157_v0.3.json: 100%|██████████| 11/11 [00:00<00:00, 322.76it/s]

COCO data converted successfully.
Results saved to /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_week9_15_06_2023_3x_regular_IMG_2157/annotations/yolo


### Convert COCO to YOLO Instance Segmentation format - Working

In [6]:
import sys
sys.path.append('/home/etaylor/code_projects')
sys.path.append('/home/etaylor/code_projects/coco2yolo')
from coco2yolo import general_json2yolo as c2y

In [7]:
import os
import shutil


annotations_dir = '/home/etaylor/code_projects/thesis/segments/etaylor_document_segmentation/annotations'
output_dir = "/home/etaylor/code_projects/thesis/segments/etaylor_document_segmentation/yolo"

# convert the RLE coco json to yolo format
c2y.convert_coco_json(
    json_dir=annotations_dir,
    use_segments=True,
)

Annotations /sise/home/etaylor/code_projects/thesis/segments/etaylor_document_segmentation/annotations/export_coco-instance_etaylor_document_segmentation_v0.1.json: 100%|██████████| 120/120 [00:04<00:00, 29.63it/s]


Fix mapping of class from -1 to 0 

In [2]:
import os
import glob

# Path to the annotation files
annotations_dir = '/home/etaylor/code_projects/thesis/segments/etaylor_stigmas_dataset/yolo/labels/export_coco-instance_etaylor_stigmas_dataset_v0.2'

# Get all .txt files in the directory
txt_files = glob.glob(os.path.join(annotations_dir, '*.txt'))

# Iterate through each file and replace -1 with 0
for txt_file in txt_files:
    with open(txt_file, 'r') as file:
        lines = file.readlines()
    
    # Replace class labels
    updated_lines = [line.replace('-1', '0', 1) for line in lines]
    
    # Write the updated content back to the file
    with open(txt_file, 'w') as file:
        file.writelines(updated_lines)

print(f"Processed {len(txt_files)} files. All '-1' class labels have been replaced with '0'.")


Processed 112 files. All '-1' class labels have been replaced with '0'.


### Use this after converting to YOLO to split to train and val splits

In [10]:
import os
import shutil
import random
from pathlib import Path

def split_and_organize_yolo_dataset(dataset_path, output_path, train_ratio=0.8, image_exts=['.jpg', '.png']):
    """
    Organize and split a YOLO dataset into train and validation sets.

    Args:
        dataset_path (str): Path to the root of the dataset containing images and labels.
        output_path (str): Path to save the organized dataset.
        train_ratio (float): Ratio of the dataset to use for training.
        image_exts (list): List of acceptable image file extensions.
    """
    # Define paths
    images_path = Path(dataset_path) / 'v0.1'
    labels_path = Path(dataset_path) / 'yolo/labels/export_coco-instance_etaylor_document_segmentation_v0.1'

    train_image_output = Path(output_path) / 'images/train'
    val_image_output = Path(output_path) / 'images/val'
    train_label_output = Path(output_path) / 'labels/train'
    val_label_output = Path(output_path) / 'labels/val'

    # Create output directories if they don't exist
    for dir_path in [train_image_output, val_image_output, train_label_output, val_label_output]:
        dir_path.mkdir(parents=True, exist_ok=True)

    # Get all image files
    image_files = [f for f in images_path.iterdir() if f.suffix in image_exts]
    random.shuffle(image_files)

    # Split the dataset
    num_train = int(len(image_files) * train_ratio)
    train_files = image_files[:num_train]
    val_files = image_files[num_train:]

    # Move or copy files to train/val
    for img_file in train_files:
        label_file = labels_path / f"{img_file.stem}.txt"
        if label_file.exists():
            shutil.copy(img_file, train_image_output / img_file.name)
            shutil.copy(label_file, train_label_output / label_file.name)

    for img_file in val_files:
        label_file = labels_path / f"{img_file.stem}.txt"
        if label_file.exists():
            shutil.copy(img_file, val_image_output / img_file.name)
            shutil.copy(label_file, val_label_output / label_file.name)

    print(f"✅ Dataset organized: {len(train_files)} train, {len(val_files)} val")

    # Return paths for registration or further use
    return {
        'train_images': train_image_output,
        'val_images': val_image_output,
        'train_labels': train_label_output,
        'val_labels': val_label_output
    }

In [11]:
dataset_path = "/home/etaylor/code_projects/thesis/segments/etaylor_document_segmentation"
output_path = "/home/etaylor/code_projects/thesis/segments/etaylor_document_segmentation/yolo_formatted"
split_and_organize_yolo_dataset(dataset_path, output_path)

✅ Dataset organized: 192 train, 48 val


{'train_images': PosixPath('/home/etaylor/code_projects/thesis/segments/etaylor_document_segmentation/yolo_formatted/images/train'),
 'val_images': PosixPath('/home/etaylor/code_projects/thesis/segments/etaylor_document_segmentation/yolo_formatted/images/val'),
 'train_labels': PosixPath('/home/etaylor/code_projects/thesis/segments/etaylor_document_segmentation/yolo_formatted/labels/train'),
 'val_labels': PosixPath('/home/etaylor/code_projects/thesis/segments/etaylor_document_segmentation/yolo_formatted/labels/val')}